# `scinum` example

In [1]:
from scinum import Number, Correlation, NOMINAL, UP, DOWN, ABS, REL

The examples below demonstrate

- [Numbers and formatting](#Numbers-and-formatting)
- [Defining uncertainties](#Defining-uncertainties)
- [Multiple uncertainties](#Multiple-uncertainties)
- [Configuration of correlations](#Configuration-of-correlations)
- [Automatic uncertainty propagation](#Automatic-uncertainty-propagation)

### Numbers and formatting

In [2]:
n = Number(1.234, 0.2)
n

<Number at 0x117020880, '1.234 +- 0.2'>

The uncertainty definition is absolute. See the examples with [multiple uncertainties](#Multiple-uncertainties) for relative uncertainty definitions.

The representation of numbers (`repr`) in jupyter notebooks uses latex-style formatting. Internally, [`Number.str()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.str) is called, which - among others - accepts a `format` argument, defaulting to `"%s"` (configurable globally or per instance via [`Number.default_format`](https://scinum.readthedocs.io/en/latest/#scinum.Number.default_format)). Let's change the format for this notebook:

In [3]:
Number.default_format = "%.2f"
n

<Number at 0x117020880, '1.23 +- 0.20'>

In [4]:
# or
n.str("%.3f")

'1.234 +- 0.200'

### Defining uncertainties

Above, `n` is defined with a single, symmetric uncertainty. Here are some basic examples to access and play it:

In [5]:
# nominal value
print(n.nominal)
print(type(n.nominal))

1.234
<class 'float'>


In [6]:
# get the uncertainty
print(n.get_uncertainty())
print(n.get_uncertainty(direction=UP))
print(n.get_uncertainty(direction=DOWN))

(0.2, 0.2)
0.2
0.2


In [7]:
# get the nominal value, shifted by the uncertainty
print(n.get())      # nominal value
print(n.get(UP))    # up variation
print(n.get(DOWN))  # down variation

1.234
1.434
1.034


In [8]:
# some more advanved use-cases:

# 1. get the multiplicative factor that would scale the nomninal value to the UP/DOWN varied ones
print("absolute factors:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

# 2. get the factor to obtain the uncertainty only (i.e., the relative unceratinty)
# (this is, of course, more useful in case of multiple uncertainties, see below)
print("\nrelative factors:")
print(n.get(UP, factor=True, diff=True))
print(n.get(DOWN, factor=True, diff=True))

absolute factors:
1.1620745542949757
0.8379254457050244

relative factors:
0.1620745542949757
0.1620745542949757


There are also a few shorthands for the above methods:

In [9]:
# __call__ is forwarded to get()
print(n())
print(n(UP))

# u() is forwarded to get_uncertainty()
print(n.u())
print(n.u(direction=UP))

1.234
1.434
(0.2, 0.2)
0.2


### Multiple uncertainties

Let's create a number that has two uncertainties: `"stat"` and `"syst"`. The `"stat"` uncertainty is asymmetric, and the `"syst"` uncertainty is relative.

In [10]:
n = Number(8848, {
    "stat": (30, 20),   # absolute +30-20 uncertainty
    "syst": (REL, 0.5),  # relative +-50% uncertainty
})
n

<Number at 0x117038a30, '8848.00 +30.00-20.00 (stat) +- 4424.00 (syst)'>

Similar to above, we can access the uncertainties and shifted values with [`get()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.get) (or `__call__`) and [`get_uncertainty()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.get_uncertainty) (or [`u()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.u)). But this time, we can distinguish between the combined (in quadrature) value or the particular uncertainty sources:

In [11]:
# nominal value as before
print(n.nominal)

# get all uncertainties (stored absolute internally)
print(n.uncertainties)

8848.0
{'stat': (30.0, 20.0), 'syst': (4424.0, 4424.0)}


In [12]:
# get particular uncertainties
print(n.u("syst"))
print(n.u("stat"))
print(n.u("stat", direction=UP))

(4424.0, 4424.0)
(30.0, 20.0)
30.0


In [13]:
# get the nominal value, shifted by particular uncertainties
print(n(UP, "stat"))
print(n(DOWN, "syst"))

# compute the shifted value for both uncertainties, added in quadrature without correlation (default but configurable)
print(n(UP))

8878.0
4424.0
13272.101716733014


As before, we can also access certain aspects of the uncertainties:

In [14]:
print("factors for particular uncertainties:")
print(n.get(UP, "stat", factor=True))
print(n.get(DOWN, "syst", factor=True))

print("\nfactors for the combined uncertainty:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

factors for particular uncertainties:
1.0033905967450272
0.5

factors for the combined uncertainty:
1.500011496014129
0.49999489062775576


We can also apply some nice formatting:

In [15]:
print(n.str())
print(n.str("%.2f"))
print(n.str("%.2f", unit="m"))
print(n.str("%.2f", unit="m", force_asymmetric=True))
print(n.str("%.2f", unit="m", scientific=True))
print(n.str("%.2f", unit="m", si=True))
print(n.str("%.2f", unit="m", style="root"))

8848.00 +30.00-20.00 (stat) +- 4424.00 (syst)
8848.00 +30.00-20.00 (stat) +- 4424.00 (syst)
8848.00 +30.00-20.00 (stat) +- 4424.00 (syst) m
8848.00 +30.00-20.00 (stat) +4424.00-4424.00 (syst) m
8.85 +0.03-0.02 (stat) +- 4.42 (syst) x 1E3 m
8.85 +0.03-0.02 (stat) +- 4.42 (syst) km
8848.00 ^{+30.00}_{-20.00} #left(stat#right) #pm 4424.00 #left(syst#right) m


### Configuration of correlations

Let's assume that we have a second measurement for the quantity `n` we defined above,

In [16]:
n

<Number at 0x117038a30, '8848.00 +30.00-20.00 (stat) +- 4424.00 (syst)'>

and we measured it with the same sources of uncertainty,

In [17]:
n2 = Number(8920, {
    "stat": (35, 15),    # absolute +35-15 uncertainty
    "syst": (REL, 0.3),  # relative +-30% uncertainty
})
n2

<Number at 0x11704e610, '8920.00 +35.00-15.00 (stat) +- 2676.00 (syst)'>

 Now, we want to compute the average measurement, including correct error propagation under consideration of sensible correlations. For more info on automatic uncertainty propagation, see the [subsequent section](#Automatic-uncertainty-propagation).
 
In this example, we want to fully correlate the *systematic* uncertainty, whereas we can treat *statistical* effects as uncorrelated. However, just wirting `(n + n2) / 2` will consider equally named uncertainty sources to be 100% correlated, i.e., both `syst` and `stat` uncertainties will be simply averaged. This is the default behavior in scinum as it is not possible (nor wise) to *guesstimate* the meaning of an uncertainty from its name.

While this approach is certainly correct for `syst`, we don't achieve the correct treatment for `stat`:

In [18]:
(n + n2) / 2

<Number at 0x11704ebb0, '8884.00 +32.50-17.50 (stat) +- 3550.00 (syst)'>

Instead, we need to define the correlation specifically for `stat`. This can be achieved in multiple ways, but the most pythonic way is to use a [`Correlation`](https://scinum.readthedocs.io/en/latest/#correlation) object.

In [19]:
(n @ Correlation(stat=0) + n2) / 2

<Number at 0x117058310, '8884.00 +23.05-12.50 (stat) +- 3550.00 (syst)'>

**Note** that the statistical uncertainty decreased as desired, whereas the systematic one remained the same.
`Correlation` objects have a default value that can be set as the first positional, yet optional parameter, and itself defaults to one.

Internally, the operation `n @ Correlation(stat=0)` (or `n * Correlation(stat=0)` in Python 2) is evaluated prior to the addition of `n2` and generates a so-called [`DeferredResult`](https://scinum.readthedocs.io/en/latest/#deferredresult). This object carries the information of `n` and the correlation over to the next operation, at which point the uncertainty propagation is eventually resolved. As usual, in situations where the operator precedence might seem unclear, it is recommended to use parentheses to structure the expression.

### Automatic uncertainty propagation

Let's continue working with the number `n` from above.

Uncertainty propagation works in a pythonic way:

In [20]:
n + 200

<Number at 0x1170583a0, '9048.00 +30.00-20.00 (stat) +- 4424.00 (syst)'>

In [21]:
n / 2

<Number at 0x117058850, '4424.00 +15.00-10.00 (stat) +- 2212.00 (syst)'>

In [22]:
n**0.5

<Number at 0x117058d60, '94.06 +0.16-0.11 (stat) +- 23.52 (syst)'>

In cases such as the last one, formatting makes a lot of sense ...

In [23]:
(n**0.5).str("%.2f")

'94.06 +0.16-0.11 (stat) +- 23.52 (syst)'

More complex operations such as `exp`, `log`, `sin`, etc, are provided on the `ops` object, which mimics Python's `math` module. The benefit of the `ops` object is that all its operations are aware of Gaussian error propagation rules.

In [24]:
from scinum import ops

# change the default format for convenience
Number.default_format = "%.3f"

# compute the log of n
ops.log(n)

<Number at 0x117064460, '9.088 +0.003-0.002 (stat) +- 0.500 (syst)'>

The propagation is actually performed simultaneously per uncertainty source.

In [25]:
m = Number(5000, {"syst": 1000})

n + m

<Number at 0x117064340, '13848.000 +30.000-20.000 (stat) +- 5424.000 (syst)'>

In [26]:
n / m

<Number at 0x117064280, '1.770 +0.006-0.004 (stat) +- 0.531 (syst)'>

As described [above](#Configuration-of-correlations), equally named uncertainty sources are assumed to be fully correlated. You can configure the correlation in operations through `Correlation` objects, or by using explicit methods on the number object.

In [27]:
# n.add(m, rho=0.5, inplace=False)

# same as
n @ Correlation(0.5) + m

<Number at 0x117064a60, '13848.000 +30.000-20.000 (stat) +- 4999.578 (syst)'>

When you set `inplace` to `True` (the default), `n` is updated inplace.

In [28]:
n.add(m, rho=0.5)
n

<Number at 0x117038a30, '13848.000 +30.000-20.000 (stat) +- 4999.578 (syst)'>